# Kaggle League of Legends competition - Data Exploration

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

In [100]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression


In [162]:
X_train = pd.read_csv('../data/participants_train.csv')
X_test = pd.read_csv('../data/participants_test.csv')
y_train = pd.read_csv('../data/train_winners.csv')

champion_mastery = pd.read_csv('../data/champion_mastery.csv')
champion = pd.read_json('../data/champion.json')
team_positions = pd.read_csv('../data/teamPositions.csv')

In [164]:
# to unpack the relevant information
champion_data = pd.json_normalize(champion['data'])

In [5]:
X_train_mastery = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId'], indicator=True)
X_train_mastery.loc[X_train_mastery['_merge'] != 'both']

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,_merge
15475,1547,200,6,4039,67,Malphite,54,NaN,NaN,NaN,NaN,left_only
17267,1726,200,8,13584,43,Zed,238,NaN,NaN,NaN,NaN,left_only
20142,2014,100,3,15354,50,Corki,42,NaN,NaN,NaN,NaN,left_only
20609,2060,200,10,15627,97,Senna,235,NaN,NaN,NaN,NaN,left_only
28330,2833,100,1,19836,34,Nasus,75,NaN,NaN,NaN,NaN,left_only
35860,3586,100,1,23525,52,KSante,897,NaN,NaN,NaN,NaN,left_only
47090,4709,100,1,28312,42,Gwen,887,NaN,NaN,NaN,NaN,left_only
53017,5301,200,8,30650,53,Cassiopeia,69,NaN,NaN,NaN,NaN,left_only
56373,5637,100,4,31856,44,Samira,360,NaN,NaN,NaN,NaN,left_only
58253,5825,100,4,8619,47,Kaisa,145,NaN,NaN,NaN,NaN,left_only


In [103]:
X_test_mastery = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId'], indicator=True)
X_test_mastery.loc[X_test_mastery['_merge'] != 'both']

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,_merge
5513,8551,100,4,41371,35,Kaisa,145,0.0,0.0,0,0.0,left_only
10156,9015,200,7,42698,43,Kayn,141,0.0,0.0,0,0.0,left_only


> NOTE on the above two: there are player - champion combinations that likely had too little games ()

In [128]:
X_test_mastery['championLevel'].value_counts()

7.0    10158
5.0     3659
6.0     1662
4.0     1534
3.0     1387
2.0     1101
1.0      497
Name: championLevel, dtype: int64

In [165]:
champion_data['key'] = champion_data['key'].astype(int)
X_train_champion = pd.merge(X_train, champion_data, how='inner', left_on='championId', right_on='key')
X_train_champion = X_train_champion.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

In [152]:
X_train_champion[['info.attack', 'info.defense', 'info.magic', 'info.difficulty']]

,info.attack,info.defense,info.magic,info.difficulty
0,4,6,7,4
1,3,4,8,5
2,2,5,8,6
3,8,2,2,6
4,1,6,8,1
...,...,...,...,...
79995,6,6,4,6
79996,7,7,4,3
79997,1,4,10,10
79998,9,2,3,6


In [161]:
team_positions

,matchId,participantId,teamPosition
0,0,1,TOP
1,0,2,JUNGLE
2,0,3,MIDDLE
3,0,4,BOTTOM
4,0,5,UTILITY
...,...,...,...
99995,9999,6,TOP
99996,9999,7,JUNGLE
99997,9999,8,MIDDLE
99998,9999,9,BOTTOM


In [192]:
X_train.tail()

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId
79995,7999,200,6,13979,595,Yorick,83
79996,7999,200,7,39643,38,Volibear,106
79997,7999,200,8,5570,498,Anivia,34
79998,7999,200,9,10228,733,Twitch,29
79999,7999,200,10,1684,574,Zilean,26


In [236]:
testing_data = pd.DataFrame()

#for i in range(0:8000):
for i in range(0,8000) :

    temp_data = pd.read_json(f'../data/train_timelines/train_timelines/timeline_{i}.json')
    temp_data = pd.json_normalize(temp_data['frames'])

    max_frame = len(temp_data) - 1

    for j in range(1, 11):
        temp_df = pd.DataFrame({'matchId': [i],
                   'participantId': [j],
                   'final_gold': temp_data[f'participantFrames.{j}.totalGold'][max_frame],
                   'final_xp': temp_data[f'participantFrames.{j}.xp'][max_frame]})

        testing_data = pd.concat([testing_data, temp_df], ignore_index = True)



In [238]:
testing_data.to_csv('../data/last_frame_values.csv', index=False)

In [167]:
timeline01 = pd.read_json('../data/train_timelines/train_timelines/timeline_0.json')

In [170]:
timeline01_test = pd.json_normalize(timeline01['frames'])

In [173]:
timeline01_test.head()

,events,timestamp,participantFrames.1.championStats.abilityHaste,participantFrames.1.championStats.abilityPower,participantFrames.1.championStats.armor,participantFrames.1.championStats.armorPen,participantFrames.1.championStats.armorPenPercent,participantFrames.1.championStats.attackDamage,participantFrames.1.championStats.attackSpeed,participantFrames.1.championStats.bonusArmorPenPercent,...,participantFrames.10.goldPerSecond,participantFrames.10.jungleMinionsKilled,participantFrames.10.level,participantFrames.10.minionsKilled,participantFrames.10.participantId,participantFrames.10.position.x,participantFrames.10.position.y,participantFrames.10.timeEnemySpentControlled,participantFrames.10.totalGold,participantFrames.10.xp
0,"[{'timestamp': 0, 'type': 'PAUSE_END'}]",0,0,0,37,0,0,25,100,0,...,0,0,1,0,10,14281,14199,0,500,0
1,"[{'itemId': 1103, 'participantId': 7, 'timesta...",60000,0,9,43,0,0,61,110,0,...,20,0,1,0,10,12292,6388,1254,500,0
2,"[{'creatorId': 9, 'timestamp': 61763, 'type': ...",120006,0,29,43,0,0,61,110,0,...,20,0,1,0,10,13483,2530,1254,523,113
3,"[{'bounty': 400, 'killStreakLength': 0, 'kille...",180026,0,9,43,0,0,61,110,0,...,20,0,2,1,10,11757,1828,4524,678,375
4,"[{'level': 2, 'participantId': 1, 'timestamp':...",240045,0,9,49,0,0,66,111,0,...,20,0,3,5,10,12586,2733,5944,936,852


In [190]:
timeline01_test['participantFrames.1.totalGold'][10]

2741

In [191]:
timeline01_test['participantFrames.1.xp'][10]

4141

In [174]:
player_1_cols = [col for col in timeline01_test.columns if '.1.' in col]

In [177]:
player_1_cols

['participantFrames.1.championStats.abilityHaste',
 'participantFrames.1.championStats.abilityPower',
 'participantFrames.1.championStats.armor',
 'participantFrames.1.championStats.armorPen',
 'participantFrames.1.championStats.armorPenPercent',
 'participantFrames.1.championStats.attackDamage',
 'participantFrames.1.championStats.attackSpeed',
 'participantFrames.1.championStats.bonusArmorPenPercent',
 'participantFrames.1.championStats.bonusMagicPenPercent',
 'participantFrames.1.championStats.ccReduction',
 'participantFrames.1.championStats.cooldownReduction',
 'participantFrames.1.championStats.health',
 'participantFrames.1.championStats.healthMax',
 'participantFrames.1.championStats.healthRegen',
 'participantFrames.1.championStats.lifesteal',
 'participantFrames.1.championStats.magicPen',
 'participantFrames.1.championStats.magicPenPercent',
 'participantFrames.1.championStats.magicResist',
 'participantFrames.1.championStats.movementSpeed',
 'participantFrames.1.championStat